Prototyped this in Colab

https://colab.research.google.com/drive/1KyXcpk3T1dDDI3f-UgMpiPzVvEAzvkUH?usp=sharing

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

import seaborn as sns

import phasespace

import vector

import tensorflow



In [ ]:
print(phasespace.__version__)

print(vector.__version__)

print(tensorflow.__version__)

In [ ]:

# Magnitude of 3-vector
def mag(p3):
  #print(p3)
  p = np.sqrt(p3[0]*p3[0] + p3[1]*p3[1] + p3[2]*p3[2])

  return p

# Invariant mass from list of 4-vectors
def invmass(p4s):
  E,px,py,pz = 0,0,0,0

  for p4 in p4s:
    E += p4[3]
    px += p4[0]
    py += p4[1]
    pz += p4[2]

  m2 = E**2 - (px**2 + py**2 + pz**2)
  if m2>=0:
    return np.sqrt(m2)
  else:
    return -np.sqrt(-m2)


# Invariablt mass from columnar representations
def invmass_cols(p4):

  E = p4[3]
  px = p4[0]
  py = p4[1]
  pz = p4[2]

  m2 = E**2 - (px**2 + py**2 + pz**2)
  m = -999*np.ones(len(E))
  mask = m2>=0
  #print(mask[mask])
  #print(mask[~mask])

  m[mask] = np.sqrt(m2[mask])
  m[~mask] = -np.sqrt(-m2[~mask])

  return m


# Calculate the opening angle between two 4-vectors
# We keep the assumed syntax from phasespace and assume the energy
# is the 4th entry (n=3) of the 4-vector
def opening_angle(p4s):
  p0mag = np.sqrt(p4s[0][0]**2 + p4s[0][1]**2 + p4s[0][2]**2)
  p1mag = np.sqrt(p4s[1][0]**2 + p4s[1][1]**2 + p4s[1][2]**2)

  dot_product = p4s[0][0]*p4s[1][0] + p4s[0][1]*p4s[1][1] + p4s[0][2]*p4s[1][2]

  theta = np.arccos(dot_product/(p0mag*p1mag))

  return theta


# Distance between two 3-vectors
def distance(v1, v2):
  dx = v1[0] - v2[0]
  dy = v1[1] - v2[1]
  dz = v1[2] - v2[2]

  d = np.sqrt(dx**2 + dy**2 + dz**2)
  return d

##################################################################


# Generate decays for the center-of-the-Earth model

In [ ]:
# Testing out np.tile to make sure I understand it

#x = np.array([1, 2, 3, 4])

#np.tile(x, (4,1))

# Generate distributions of muons

Loop over assumed values for
* Dark matter candidate mass ($M_{DM}$)
* Dark photon candidate mass ($M_{A}$)

Assume that the dark photons are coming straight *up* from underground. So they only have the $z$ component of momentum.

In [ ]:
# I'm going to store everthing in a dataframe, but to start, I create
# a dictionary to hold all the values

decays = {}

# Mass of DM and dark photon candidates
decays['M_DM'] = []
decays['M_A'] = []

# The lab/Earth-frame 4-vectors for the final-state muons
#
# We're going to assume that there is "another" dark photon
# on the other side of the Earth, but we will ignore that

decays['px_mu1'] = []
decays['py_mu1'] = []
decays['pz_mu1'] = []
decays['e_mu1'] = []

decays['px_mu2'] = []
decays['py_mu2'] = []
decays['pz_mu2'] = []
decays['e_mu2'] = []


# Dark photon masses to try (GeV/c^2)
MASS_A = [.250,1,5]

# DM masses to try (GeV/c^2)
DM_MASSES = [10,100,1000]

MUON_MASS = 0.10511

# Helper lists to store the
#thetas = []
#pmags = []
#muon_p = []

nevents_to_generate = 100000

#pmags_GeV = [10,100,1000]

for MASS_A in MASS_A:
  for pmag in DM_MASSES:

    # Because the mass of the dark photon is assumed to be much less that our dark matter
    # candidates, we assume that the dark photon aquires the same kinetic energy / momentum
    # as the mass of one of the DM candidates particles.
    #
    # I'm making some approximations here and my assumptions are probably least
    # precise for M_DM = 10 and M_A = 5

    print(MASS_A,pmag)

    # We boost the final state muons, assuming that they originate from the dark photon
    # traveling straight up

    boost_vector = np.array([0,0, pmag, np.sqrt(pmag**2 + MASS_A**2)])
    boost_vectors = np.tile(boost_vector, (nevents_to_generate,1))

    weights, particles = phasespace.nbody_decay(MASS_A,
                                                [MUON_MASS, MUON_MASS]).generate(n_events=nevents_to_generate, boost_to=boost_vectors)

    # This syntax has been superceded in the latest version
    #weights, particles = phasespace.nbody_decay(MASS_A,
    #                                            [MUON_MASS, MUON_MASS]).generate(n_events=nevents_to_generate, boost_to=np.array([0,0, pmag, np.sqrt(pmag**2 + MASS_A**2)]))

    # Don't boost the muons
    #weights, particles = phasespace.nbody_decay(MASS_A,
    #                                            [MUON_MASS, MUON_MASS]).generate(n_events=nevents_to_generate )

    # Pull out the 4-vecs for the final-state muons
    p0 = particles['p_0'][:].numpy().T
    p1 = particles['p_1'][:].numpy().T

    #data[MASS_A][pmag] = [p0.T, p1.T]
    decays['M_DM'] += (pmag*np.ones(nevents_to_generate)).tolist()
    decays['M_A'] += (MASS_A*np.ones(nevents_to_generate)).tolist()

    # Fill our dictionary.
    # Note that the energy is the 4th entry of their 4-vector
    decays['px_mu1'] += p0[0].tolist()
    decays['py_mu1'] += p0[1].tolist()
    decays['pz_mu1'] += p0[2].tolist()
    decays['e_mu1'] +=  p0[3].tolist()

    decays['px_mu2'] += p1[0].tolist()
    decays['py_mu2'] += p1[1].tolist()
    decays['pz_mu2'] += p1[2].tolist()
    decays['e_mu2'] +=  p1[3].tolist()

# Make a dataframe for these decays
dfdec1 = pd.DataFrame.from_dict(decays)

# Generate a few other entries

# pmag, theta (degrees), phi for the two muons
px1 = dfdec1['px_mu1'].values
py1 = dfdec1['py_mu1'].values
pz1 = dfdec1['pz_mu1'].values
e1 = dfdec1['e_mu1'].values

pmag1 = mag([px1,py1,pz1])
theta1 = np.rad2deg(np.arccos(pz1/pmag1))
phi1 = np.arctan2(py1,pz1)

dfdec1['pmag1'] = pmag1
dfdec1['theta1'] = theta1
dfdec1['phi1'] = phi1

px2 = dfdec1['px_mu2'].values
py2 = dfdec1['py_mu2'].values
pz2 = dfdec1['pz_mu2'].values
e2 = dfdec1['e_mu2'].values

pmag2 = mag([px2,py2,pz2])
theta2 = np.rad2deg(np.arccos(pz2/pmag2))
phi2 = np.arctan2(py2,pz2)

dfdec1['pmag2'] = pmag2
dfdec1['theta2'] = theta2
dfdec1['phi2'] = phi2

# Opening angle between the two muons
p4s = [[px1,py1,pz1,e1], [px2,py2,pz2,e2]]
thetas = np.rad2deg(opening_angle(p4s))
dfdec1['opening angle'] = thetas

dfdec1.sample(5)

# Plots

In [ ]:
dfdec1['opening angle']

In [ ]:
# Inspect our dataframe
mask = (dfdec1['M_DM']==100) & (dfdec1['M_A']==0.25)

#dfdec1[mask]['opening angle']
dfdec1[mask]['pmag1']

In [ ]:
palette = sns.color_palette("bright",3)

g = sns.displot(dfdec1, x='opening angle', hue='M_DM', col='M_A', bins=50, palette=palette)

# Legend texts
g.legend.get_title().set_fontsize(20)
g.legend.set_title(r"M$_{DM}$ (GeV/c$^2$)")
for text in g.legend.texts:
    text.set_fontsize(20)

plt.savefig("signal_opening_angle_000.png")

In [ ]:
g = sns.displot(dfdec1, x='opening angle', col='M_DM', hue='M_A', bins=50, palette=palette)

# Legend texts
g.legend.get_title().set_fontsize(20)
g.legend.set_title(r"M$_{A'}$ (MeV)")
for text in g.legend.texts:
    text.set_fontsize(20)

axes = g.axes.flat
axes[0].set_title(r'M$_{DM}$ = 10 GeV', fontsize=20)
axes[1].set_title(r'M$_{DM}$ = 100 GeV', fontsize=20)
axes[2].set_title(r'M$_{DM}$ = 1000 GeV', fontsize=20)

plt.savefig("signal_opening_angle_001.png")

In [ ]:
g = sns.displot(dfdec1, x='theta2', col='M_DM', hue='M_A', bins=50, palette=palette, fill=True)#, height=5, aspect=1.5, fill=True)

#g.set(ylim=(0,50))

# Legend texts
g.legend.get_title().set_fontsize(20)
g.legend.set_title(r"M$_{A'}$ (MeV)")
for text in g.legend.texts:
    text.set_fontsize(20)

axes = g.axes.flat
axes[0].set_title(r'M$_{DM}$ = 10 GeV', fontsize=20)
axes[1].set_title(r'M$_{DM}$ = 100 GeV', fontsize=20)
axes[2].set_title(r'M$_{DM}$ = 1000 GeV', fontsize=20)

plt.savefig("signal_polar_angle_000.png")

In [ ]:
g = sns.displot(dfdec1, x='pmag1', col='M_DM', hue='M_A', bins=50, palette=palette, facet_kws=dict(sharey=False, sharex=False))

for ax in g.axes.flat:
  ax.ticklabel_format(axis="x", style="plain", scilimits=(0,0))

axes = g.axes.flat
axes[0].set_title(r'M$_{DM}$ = 10 GeV', fontsize=20)
axes[1].set_title(r'M$_{DM}$ = 100 GeV', fontsize=20)
axes[2].set_title(r'M$_{DM}$ = 1000 GeV', fontsize=20)

# Legend texts
g.legend.get_title().set_fontsize(20)
g.legend.set_title(r"M$_{A'}$ (GeV)")
for text in g.legend.texts:
    text.set_fontsize(20)

plt.savefig("signal_pmag_org_000.png")

In [ ]:
# Make the plots for just 1 assumed mass of the dark photon
mask = dfdec1['M_A']==0.25
g = sns.displot(dfdec1[mask], x='pmag1', col='M_DM', hue='M_A', bins=50, palette=palette, common_bins=False, facet_kws=dict(sharey=False, sharex=False))

for ax in g.axes.flat:
  ax.ticklabel_format(axis="x", style="plain", scilimits=(0,0))

axes = g.axes.flat
axes[0].set_title(r'M$_{DM}$ = 10 GeV', fontsize=20)
axes[1].set_title(r'M$_{DM}$ = 100 GeV', fontsize=20)
axes[2].set_title(r'M$_{DM}$ = 1000 GeV', fontsize=20)

# Legend texts
g.legend.get_title().set_fontsize(20)
g.legend.set_title(r"M$_{A'}$ (GeV)")
for text in g.legend.texts:
    text.set_fontsize(20)

plt.savefig("signal_pmag_001.png")